## FY23 FTA Bus and Low- and No-Emission Grant Awards Analysis

<b>GH issue:</b> 
* Research Request - Bus Procurement Costs & Awards #897

<b>Data source(s):</b> 
1. https://www.transit.dot.gov/funding/grants/fy23-fta-bus-and-low-and-no-emission-grant-awards
2. https://storymaps.arcgis.com/stories/022abf31cedd438b808ec2b827b6faff

<b>Definitions:</b>  
* <u>Grants for Buses and Bus Facilities Program:</u>
    * 49 U.S.C. 5339(b)) makes federal resources available to states and direct recipients to replace, rehabilitate and purchase buses and related equipment and to construct bus-related facilities, including technological changes or innovations to modify low or no emission vehicles or facilities. Funding is provided through formula allocations and competitive grants. 
<br><br>
* <u>Low or No Emission Vehicle Program:</u>
    * 5339(c) provides funding to state and local governmental authorities for the purchase or lease of zero-emission and low-emission transit buses as well as acquisition, construction, and leasing of required supporting facilities.


In [1]:
# import shared_utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm

# set_option to increase max rows displayed to 200, to see entire df in 1 go/
pd.set_option("display.max_rows", 300)

## Reading in raw data from gcs

In [2]:
df = pd.read_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/data-analyses_bus_procurement_cost_fta_press_release_data_csv.csv"
)

## Data Cleaning
1. snake-case column names
2. remove currency formatting from funding column (with $ and , )
3. seperate text from # of bus col (split at '(')
    a. trim spaces in new col
    b. get rid of () characters in new col
4. trim spaces in other columns
5. exnamine column values and replace/update as needed
6. create new columns for bus size type and prop type


In [3]:
# fucntions to clean up dataframe and df columns
def snake_case(df):
    df.columns=df.columns.str.lower()
    df.columns=df.columns.str.replace(" ", "_")
    df.columns=df.columns.str.strip()

def fund_cleaner(df,column):
    df[column]= df[column].str.replace("$", "")
    df[column]= df[column].str.replace(",", "")
    df[column]= df[column].str.strip()



### Dataframe cleaning

In [4]:
#snake case function to Df
snake_case(df)

### Column Cleaning

#### propulsion_type

In [5]:
#rename col to propulsion category
df=df.rename(columns={'propulsion_type':'propulsion_category'})

In [6]:
# make values in prop_cat col lower case and remove spaces
df["propulsion_category"] = df["propulsion_category"].str.lower()
df["propulsion_category"] = df["propulsion_category"].str.replace(" ", "")

In [7]:
df.head(3)

,state,project_sponsor,project_title,description,funding,approx_#_of_buses,project_type,propulsion_category,area_served,congressional_districts,fta_region,bus/low-no_program
0,DC,Washington Metropolitan Area Transit Authority...,Battery-Electric Metrobus Procurement and Elec...,WMATA will receive funding to convert its Cind...,"$104,000,000",100(beb),bus/chargers,zero,Large Urban,DC-001 ; MD-004 ; MD-008 ; VA-008 ; VA-011,3,Low-No
1,TX,Dallas Area Rapid Transit (DART),DART CNG Bus Fleet Modernization Project,Dallas Area Rapid Transit will receive funding...,"$103,000,000",90 (estimated-CNG buses),bus,low,Large Urban,TX-003 ; TX-004 ; TX-005 ; TX-006 ; TX-024 ; T...,6,Low-No
2,PA,Southeastern Pennsylvania Transportation Autho...,SEPTA Zero-Emission Bus Transition Facility Sa...,The Southeastern Pennsylvania Transportation A...,"$80,000,000",0,facility,zero,Large Urban,PA-002 ; PA-003 ; PA-004 ; PA-005,3,Low-No


#### funding

In [8]:
fund_cleaner(df,'funding')

/tmp/ipykernel_642/4154908731.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[column]= df[column].str.replace("$", "")


In [9]:
df["funding"] = df["funding"].astype("int64")

In [10]:
df.columns

Index(['state', 'project_sponsor', 'project_title', 'description', 'funding',
       'approx_#_of_buses', 'project_type', 'propulsion_category',
       'area_served', 'congressional_districts', 'fta_region',
       'bus/low-no_program'],
      dtype='object')

#### approx_#_of_buses to bus_count and prop_type

In [11]:
# test of removing the spaces first in # of bus colum, THEN split by (
df["approx_#_of_buses"] = df["approx_#_of_buses"].str.replace(" ", "")

In [12]:
# spliting the # of buses column into 2, using the ( char as the delimiter
df[["bus_count", "prop_type"]] = df["approx_#_of_buses"].str.split(pat="(", n=1, expand=True)

In [13]:
df.head(3)

,state,project_sponsor,project_title,description,funding,approx_#_of_buses,project_type,propulsion_category,area_served,congressional_districts,fta_region,bus/low-no_program,bus_count,prop_type
0,DC,Washington Metropolitan Area Transit Authority...,Battery-Electric Metrobus Procurement and Elec...,WMATA will receive funding to convert its Cind...,104000000,100(beb),bus/chargers,zero,Large Urban,DC-001 ; MD-004 ; MD-008 ; VA-008 ; VA-011,3,Low-No,100,beb)
1,TX,Dallas Area Rapid Transit (DART),DART CNG Bus Fleet Modernization Project,Dallas Area Rapid Transit will receive funding...,103000000,90(estimated-CNGbuses),bus,low,Large Urban,TX-003 ; TX-004 ; TX-005 ; TX-006 ; TX-024 ; T...,6,Low-No,90,estimated-CNGbuses)
2,PA,Southeastern Pennsylvania Transportation Autho...,SEPTA Zero-Emission Bus Transition Facility Sa...,The Southeastern Pennsylvania Transportation A...,80000000,0,facility,zero,Large Urban,PA-002 ; PA-003 ; PA-004 ; PA-005,3,Low-No,0,None


#### bus_count

In [14]:
# function to find the row index of a specific value and column in a dataframe
def find_loc(data, col, val):
    x = data.loc[data[col] == val].index[0]
    return x

In [15]:
loc1 = find_loc(df, "bus_count", "56estimated-cutawayvans")
loc2 = find_loc(df, "bus_count", "12batteryelectric")

In [16]:
display(loc1, loc2)

58

32

In [17]:
# editing the values of the bus count col at specific location
# syntax, look at ## index, look at XX column
df.loc[58, "bus_count"] = 56
df.loc[32, "bus_count"] = 12

In [18]:
# updating values again for bus_desc. same location
df.loc[58, "prop_type"] = "estimated-cutaway vans (PM- award will not fund 68 buses)"
df.loc[32, "prop_type"] = "battery electric"

In [19]:
# bus count for row 12 needs to be adjusted to 31 instead of 15
df.loc[12, "bus_count"] = 31

In [20]:
# confirming the change
df.loc[12]

state                                                                     NC
project_sponsor            City of Charlotte - Charlotte Area Transit System
project_title              Charlotte Area Transit System's Sustainable Fl...
description                The city of Charlotte will receive funding to ...
funding                                                             30890413
approx_#_of_buses                                   15(Electric)\n16(Hybrid)
project_type                                      Bus / Chargers / Equipment
propulsion_category                                                 zero/low
area_served                                                      Large Urban
congressional_districts           NC-008 ; NC-012 ; NC-013 ; NC-014 ; SC-005
fta_region                                                                 4
bus/low-no_program                                                       Bus
bus_count                                                                 31

#### project_type

In [21]:
# using str.lower() on project type
df["project_type"] = df["project_type"].str.lower()
# using str.lower() on project type
df["project_type"] = df["project_type"].str.replace(" ", "")

In [22]:
# some values still need to get adjusted. will use a short dictionary to fix
new_type = {
    "\tbus/facility": "bus/facility",
    "bus/facilitiy": "bus/facility",
    "facilities": "facility",
}

In [23]:
# using replace() with the dictionary to replace keys in project type col
# syntax df.replace({'bus_desc': new_dict}, inplace=True)
df.replace({"project_type": new_type}, inplace=True)

In [24]:
df.project_type.unique()

array(['bus/chargers', 'bus', 'facility', 'bus/chargers/equipment',
       'facility/chargers', 'bus/facility', 'bus/facility/chargers',
       'chargers', 'bus/chargers/other', 'bus/facility/equipment',
       'bus/equipment', 'bus/facility/chargers/equipment',
       'bus/facility/other', 'facility/chargers/equipment',
       'facility/equipment', 'chargers/equipment', 'bus/other',
       'bus/facility/equipment/other'], dtype=object)

#### `prop_type`

In [25]:
# clearning the bus desc/prop_type col.
# removing the )
df["prop_type"] = df["prop_type"].str.replace(")", "")

/tmp/ipykernel_642/2225036280.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["prop_type"] = df["prop_type"].str.replace(")", "")


In [26]:
df["prop_type"].unique()

array(['beb', 'estimated-CNGbuses', None, 'zero-emission', 'cngbuses',
       'BEBs', 'Electric\n16(Hybrid', 'FCEB', 'Electric',
       'FuelCellElectric', 'CNG', 'FuelCell', 'hybrid', 'BEB',
       'battery electric', 'lowemissionCNG', 'cng',
       'BEBsparatransitbuses', 'hybridelectric', 'zeroemissionbuses',
       'dieselelectrichybrids', 'hydrogenfuelcell',
       '2BEBsand4HydrogenFuelCellBuses', '4fuelcell/3CNG',
       'estimated-cutaway vans (PM- award will not fund 68 buses',
       'hybridelectricbuses', 'CNGfueled', 'zeroemissionelectric',
       'hybridelectrics', 'dieselandgas', 'diesel-electrichybrids',
       'propane', 'electric', 'diesel-electric', 'propanebuses',
       '1:CNGbus;2cutawayCNGbuses', 'zeroemission',
       'propanedpoweredvehicles'], dtype=object)

In [27]:
# stripping the values in the bus desc col
df["prop_type"] = df["prop_type"].str.strip()

In [28]:
# creating a dictionary to add spaces back to the values
new_dict = {
    "beb": "BEB",
    "estimated-CNGbuses": "estimated-CNG buses",
    "cngbuses": "CNG buses",
    "BEBs": "BEB",
    "Electric\n16(Hybrid": "15 electic, 16 hybrid",
    "FuelCellElectric": "fuel cell electric",
    "FuelCell": "fuel cell",
    "lowemissionCNG": "low emission CNG",
    "cng": "CNG",
    "BEBsparatransitbuses": "BEBs paratransit buses",
    "hybridelectric": "hybrid electric",
    "zeroemissionbuses": "zero emission buses",
    "dieselelectrichybrids": "diesel electric hybrids",
    "hydrogenfuelcell": "hydrogen fuel cell",
    "2BEBsand4HydrogenFuelCellBuses": "2 BEBs and 4 hydrogen fuel cell buses",
    "4fuelcell/3CNG": "4 fuel cell / 3 CNG",
    "hybridelectricbuses": "hybrid electric buses",
    "CNGfueled": "CNG fueled",
    "zeroemissionelectric": "zero emission electric",
    "hybridelectrics": "hybrid electrics",
    "dieselandgas": "diesel and gas",
    "diesel-electrichybrids": "diesel-electric hybrids",
    "propanebuses": "propane buses",
    "1:CNGbus;2cutawayCNGbuses": "1:CNGbus ;2 cutaway CNG buses",
    "zeroemission": "zero emission",
    "propanedpoweredvehicles": "propaned powered vehicles",
}

In [29]:
# using new dictionary to replace values in the bus desc col
df.replace({"prop_type": new_dict}, inplace=True)

In [30]:
bus_dict = {
    "BEBs paratransit buses": "BEB",
    "CNG buses": "CNG",
    "CNG fueled": "CNG",
    "Electric": "electrc (not specified)",
    "battery electric": "BEB",
    "diesel electric hybrids": "diesel-electric hybrids",
    "diesel-electric": "diesel-electric hybrids",
    "electric": "electrc (not specified)",
    "estimated-CNG buses": "CNG",
    "fuel cell": "FCEB",
    "fuel cell electric": "FCEB",
    "hybrid": "hybrid electric",
    "hybrid electric buses": "hybrid electric",
    "hybrid electrics": "hybrid electric",
    "low emission CNG": "CNG",
    "propane buses": "propane",
    "propaned powered vehicles": "propane",
    "zero emission": "zero-emission bus (not specified)",
    "zero emission buses": "zero-emission bus (not specified)",
    "zero emission electric": "zero-emission bus (not specified)",
    "zero-emission": "zero-emission bus (not specified)",
}

In [31]:
# repalcing values in bus_desc with bus_dict dictionary
df.replace({"prop_type": bus_dict}, inplace=True)

In [32]:
#check work
list(df.prop_type.unique())

['BEB',
 'CNG',
 None,
 'zero-emission bus (not specified)',
 '15 electic, 16 hybrid',
 'FCEB',
 'electrc (not specified)',
 'hybrid electric',
 'diesel-electric hybrids',
 'hydrogen fuel cell',
 '2 BEBs and 4 hydrogen fuel cell buses',
 '4 fuel cell / 3 CNG',
 'estimated-cutaway vans (PM- award will not fund 68 buses',
 'diesel and gas',
 'propane',
 '1:CNGbus ;2 cutaway CNG buses']

In [33]:
df.head()

,state,project_sponsor,project_title,description,funding,approx_#_of_buses,project_type,propulsion_category,area_served,congressional_districts,fta_region,bus/low-no_program,bus_count,prop_type
0,DC,Washington Metropolitan Area Transit Authority...,Battery-Electric Metrobus Procurement and Elec...,WMATA will receive funding to convert its Cind...,104000000,100(beb),bus/chargers,zero,Large Urban,DC-001 ; MD-004 ; MD-008 ; VA-008 ; VA-011,3,Low-No,100,BEB
1,TX,Dallas Area Rapid Transit (DART),DART CNG Bus Fleet Modernization Project,Dallas Area Rapid Transit will receive funding...,103000000,90(estimated-CNGbuses),bus,low,Large Urban,TX-003 ; TX-004 ; TX-005 ; TX-006 ; TX-024 ; T...,6,Low-No,90,CNG
2,PA,Southeastern Pennsylvania Transportation Autho...,SEPTA Zero-Emission Bus Transition Facility Sa...,The Southeastern Pennsylvania Transportation A...,80000000,0,facility,zero,Large Urban,PA-002 ; PA-003 ; PA-004 ; PA-005,3,Low-No,0,None
3,LA,New Orleans Regional Transit Authority,Accelerating Zero-Emissions Mobility for a Res...,The New Orleans Regional Transit Authority wil...,71439261,20(zero-emission),bus/chargers/equipment,zero,Large Urban,LA-002 ; LA-001,6,Low-No,20,zero-emission bus (not specified)
4,NJ,New Jersey Transit Corporation,Hilton Bus Garage Modernization,New Jersey Transit will receive funding to mod...,47000000,0,facility/chargers,zero,Large Urban,nj-011,2,Bus,0,None


### Need new column for `bus size type` via list and function
cutaway, 40ft etc

In [36]:
list(df.columns)

['state',
 'project_sponsor',
 'project_title',
 'description',
 'funding',
 'approx_#_of_buses',
 'project_type',
 'propulsion_category',
 'area_served',
 'congressional_districts',
 'fta_region',
 'bus/low-no_program',
 'bus_count',
 'prop_type']

In [37]:
bus_size = [
    "standard",
    "40 foot",
    "40-foot",
    "40ft",
    "articulated",
    "cutaway",
]

In [38]:
# Function to match keywords
def find_bus_size_type(description):
    for keyword in bus_size:
        if keyword in description.lower():
            return keyword
    return "not specified"

In [39]:
# new column called bus size type based on description column
df["bus_size_type"] = df["description"].apply(find_bus_size_type)

In [47]:
#check work
display(
    df.head(3),
    df.bus_size_type.unique(),
    df.shape
)

,state,project_sponsor,project_title,description,funding,approx_#_of_buses,project_type,propulsion_category,area_served,congressional_districts,fta_region,bus/low-no_program,bus_count,prop_type,bus_size_type
0,DC,Washington Metropolitan Area Transit Authority...,Battery-Electric Metrobus Procurement and Elec...,WMATA will receive funding to convert its Cind...,104000000,100(beb),bus/chargers,zero,Large Urban,DC-001 ; MD-004 ; MD-008 ; VA-008 ; VA-011,3,Low-No,100,BEB,not specified
1,TX,Dallas Area Rapid Transit (DART),DART CNG Bus Fleet Modernization Project,Dallas Area Rapid Transit will receive funding...,103000000,90(estimated-CNGbuses),bus,low,Large Urban,TX-003 ; TX-004 ; TX-005 ; TX-006 ; TX-024 ; T...,6,Low-No,90,CNG,not specified
2,PA,Southeastern Pennsylvania Transportation Autho...,SEPTA Zero-Emission Bus Transition Facility Sa...,The Southeastern Pennsylvania Transportation A...,80000000,0,facility,zero,Large Urban,PA-002 ; PA-003 ; PA-004 ; PA-005,3,Low-No,0,None,not specified


array(['not specified', 'cutaway'], dtype=object)

(130, 15)

## Exporting cleaned data to GCS

In [43]:
# saving to GCS as csv
df.to_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_bus_cost_clean.csv"
)

## Reading in cleaned data from GCS

In [44]:
bus_cost = pd.read_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_bus_cost_clean.csv"
)

In [48]:
# drop unnessary columns
bus_cost = bus_cost.drop(["Unnamed: 0", "congressional_districts"], axis=1)

In [45]:
# confirming cleaned data shows as expected.
display(bus_cost.shape, type(bus_cost), bus_cost.columns)

(130, 16)

pandas.core.frame.DataFrame

Index(['Unnamed: 0', 'state', 'project_sponsor', 'project_title',
       'description', 'funding', 'approx_#_of_buses', 'project_type',
       'propulsion_category', 'area_served', 'congressional_districts',
       'fta_region', 'bus/low-no_program', 'bus_count', 'prop_type',
       'bus_size_type'],
      dtype='object')

In [51]:
bus_cost['project_sponsor'].sort_values(ascending=True).unique()

array(['AUTORIDAD METROPOLITANA DE AUTOBUSES (PRMBA)',
       'Alabama Agricultural and Mechanical University',
       'Alameda-Contra Costa Transit District',
       'Berkshire Regional Transit Authority', 'Brazos Transit District',
       'Cape Fear Public Transportation Authority',
       'Central Oklahoma Transportation and Parking Authority (COTPA), dba EMBARK',
       'Champaign-Urbana Mass Transit District',
       'Charleston Area Regional Transportation Authority',
       'City Of Tallahassee', 'City of Albuquerque', 'City of Alexandria',
       'City of Anaheim', 'City of Bangor, Community Connector',
       'City of Beaumont', 'City of Beloit', 'City of Brownsville',
       'City of Charlotte - Charlotte Area Transit System',
       'City of Colorado Springs dba Mountain Metropolitan Transit',
       'City of Dubuque', 'City of Hattiesburg', 'City of High Point',
       'City of Iowa City', 'City of Jonesboro, Arkansas',
       'City of Knoxville', 'City of Madison', 'City o

## DEPRECATED - Data Analysis
actual data analysis and summary stats exist in the `cost_per_bus_analysis.ipynb` notebook

### Cost per Bus, per Transit Agency dataframe

In [ ]:
only_bus = bus_cost[bus_cost["bus_count"] > 0]
only_bus.head()

In [ ]:
cost_per_bus = (
    only_bus.groupby("project_sponsor")
    .agg({"funding": "sum", "bus_count": "sum"})
    .reset_index()
)

In [ ]:
cost_per_bus["cost_per_bus"] = (
    cost_per_bus["funding"] / cost_per_bus["bus_count"]
).astype("int64")

In [ ]:
cost_per_bus.dtypes

In [ ]:
cost_per_bus

In [ ]:
## export cost_per_bus df to gcs
cost_per_bus.to_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_cost_per_bus.csv"
)

### Cost per bus, stats analysis

In [ ]:
# read in fta cost per bus csv
cost_per_bus = pd.read_csv(
    "gs://calitp-analytics-data/data-analyses/bus_procurement_cost/fta_cost_per_bus.csv"
)

In [ ]:
display(cost_per_bus.shape, cost_per_bus.head())

### Initial Summary Stats

### Summary Stats

In [ ]:
# top level alanysis

bus_cost.agg({"project_title": "count", "funding": "sum", "bus_count": "sum"})

In [ ]:
# start of agg. by project_type

bus_cost.groupby("project_type").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
)

In [ ]:
# agg by program

bus_cost.groupby("bus/low-no_program").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
)

In [ ]:
# agg by state, by funding
bus_cost.groupby("state").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
).sort_values(by="funding", ascending=False)

### Projects with bus purchases

In [ ]:
# df of only projects with a bus count
only_bus = bus_cost[bus_cost["bus_count"] > 0]

In [ ]:
display(only_bus.shape, only_bus.columns)

In [ ]:
# agg by propulsion type
only_bus["propulsion_type"].value_counts()

In [ ]:
only_bus.project_type.value_counts()

In [ ]:
# of the rows with bus_count >1, what are the project types?
bus_agg = only_bus.groupby("project_type").agg(
    {"project_type": "count", "funding": "sum", "bus_count": "sum"}
)

In [ ]:
# new column that calculates `cost per bus`
bus_agg["cost_per_bus"] = (bus_agg["funding"] / bus_agg["bus_count"]).astype("int64")

In [ ]:
bus_agg

### Projects with no buses

In [ ]:
no_bus = bus_cost[bus_cost["bus_count"] < 1]

In [ ]:
no_bus["project_type"].value_counts()

### Overall Summary

In [ ]:
project_count = bus_cost.project_title.count()
fund_sum = bus_cost.funding.sum()
bus_count_sum = bus_cost.bus_count.sum()
overall_cost_per_bus = (fund_sum) / (bus_count_sum)
bus_program_count = bus_cost["bus/low-no_program"].value_counts()

projects_with_bus = only_bus.project_title.count()
projects_with_bus_funds = only_bus.funding.sum()
cost_per_bus = (only_bus.funding.sum()) / (bus_count_sum)

In [ ]:
summary = f"""
Top Level observation:
- {project_count} projects awarded
- ${fund_sum:,.2f} dollars awarded
- {bus_count_sum} buses to be purchased
- ${overall_cost_per_bus:,.2f} overall cost per bus

Projects have some mix of buses, facilities and equipment. Making it difficult to disaggregate actual bus cost.

Of the {project_count} projects awarded, {projects_with_bus} projects inlcuded buses. The remainder were facilities, chargers and equipment

Projects with buses purchases:
- {projects_with_bus} projects
- ${projects_with_bus_funds:,.2f} awarded to purchases buses
- ${cost_per_bus:,.2f} cost per bus
"""

In [ ]:
print(summary)

In [ ]:
# Assuming your DataFrame is named df
cost_per_bus_values = cost_per_bus["cost_per_bus"]

# Calculate mean and standard deviation
mean_value = cost_per_bus_values.mean()
std_deviation = cost_per_bus_values.std()

# Plot histogram
plt.hist(cost_per_bus_values, bins=30, color="skyblue", edgecolor="black", alpha=0.7)

# Add vertical lines for mean and standard deviation
plt.axvline(mean_value, color="red", linestyle="dashed", linewidth=2, label="Mean")
plt.axvline(
    mean_value + std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean + 1 Std Dev",
)
plt.axvline(
    mean_value - std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean - 1 Std Dev",
)

# Set labels and title
plt.xlabel("cost_per_bus")
plt.ylabel("Frequency")
plt.title("Histogram of cost_per_bus with Mean and Std Dev Lines")
plt.legend()

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd

# Assuming your DataFrame is named df
cost_per_bus_values = cost_per_bus["cost_per_bus"]

# Calculate mean and standard deviation
mean_value = cost_per_bus_values.mean()
std_deviation = cost_per_bus_values.std()

# Plot histogram
plt.hist(cost_per_bus_values, bins=20, color="skyblue", edgecolor="black", alpha=0.7)

# Add vertical lines for mean and standard deviation
plt.axvline(mean_value, color="red", linestyle="dashed", linewidth=2, label="Mean")
plt.axvline(
    mean_value + std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean + 1 Std Dev",
)
plt.axvline(
    mean_value - std_deviation,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label="Mean - 1 Std Dev",
)

# Set labels and title
plt.xlabel("Cost per Bus (USD)")
plt.ylabel("Frequency")
plt.title("Histogram of Cost per Bus with Mean and Std Dev Lines")
plt.legend()

# Format x-axis ticks as USD
plt.gca().xaxis.set_major_formatter(mticker.StrMethodFormatter("${x:,.0f}"))

# Show the plot
plt.show()